# SIERPINSKI

A musical hommage to the Sierpinski triangle using a recursive generator to create self-similar melodies based on a set of tones representing the "sides" of a triangle. 

<hr style="height:1px;color:gray">

Notebook imports:

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from musx import version, setmidiplayer, playfile, Score, Note, Seq, MidiFile, keynum
print(f'musx version:', version)

This notebook generates MIDI files and automatically plays them using [fluidsynth](https://www.fluidsynth.org/download/) and the [MuseScore_General.sf3](https://ftp.osuosl.org/pub/musescore/soundfont/MuseScore_General) sound font. See [INSTALL.md](https://github.com/musx-admin/musx/blob/main/INSTALL.md) for how to install a terminal-based MIDI player to use with musx.  If you don't have a player installed you can access the output files in the same directory as this notebook:

In [ ]:
setmidiplayer("fluidsynth -iq -g1 /usr/local/sf/MuseScore_General.sf3")
print('OK!')

The recusive generator:

In [ ]:
def sierpinski(score, tone, shape, trans, levels, dur, amp):
    """
    Generates a melodic shape based on successive transpositions (levels) of
    itself. 
    
    Parameters
    ----------
    score : Score
        The musical score to add events to.
    tone : keynum
        The melodic tone on which to base the melody for the current level.
    shape : list
        A list of intervals defining the melodic shape. 
    levels : int
        The number of levels the melody should be reproduced on. 
    dur : int | float
        The duration of the process.
    amp : float
        The amplitude of the process.
    """
    num = len(shape)
    for i in shape:
        k = tone + i
        # play current tone in melody
        n = Note(time=score.now, duration=dur, pitch=min(k,127), amplitude=amp, instrument=0)
        score.add(n)
        if (levels > 1):
            # sprout melody on tone at next level
            score.compose(sierpinski(score, (k + trans), shape,
                        trans, levels - 1, dur / num,  amp))
        yield dur
print(f"sierpinski: {sierpinski}")

Define track0 to be a midi meta track that holds tempo, midi instrument assignments, micro tuning, etc.:

In [ ]:
track0 = MidiFile.metatrack()
print(f"track0: {track0}")

Track1 will hold the composition:

In [ ]:
track1 = Seq()
print(f"track1: {track1}")

Create a score and pass it track1 to receive the output midi event data:

In [ ]:
score = Score(out=track1)
print(f"score: {score}")

Create the composition. Specify levels and melody length with care! The number of musical events generated by this recursive process is exponentially related to the length of the melody and the number of levels. For example the active `compose()` statement generates only 120 events but the two versions that are commented out will generate 726 and 2728 events, respectively!

In [ ]:
score.compose(sierpinski(score, keynum('a0'), [0, 7, 5], 12, 4, 3, .5))
#score.compose(sierpinski(score, keynum('a0'), [0, 7, 5], 8, 5, 7, .5))
#score.compose(sierpinski(score, keynum('a0'), [0, -1, 2, 13], 12, 5, 24, .5))
print("OK!")

Write the tracks to a midi file in the current directory and play it if possible:

In [ ]:
file = MidiFile("sierpinski.mid", [track0, track1]).write()
print(f"Wrote '{file.pathname}'")
playfile(file.pathname)